# Error Analysis for HTR

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/olaflaitinen/Thulium/blob/main/examples/03_error_analysis.ipynb)
[![PyPI](https://img.shields.io/pypi/v/thulium-htr)](https://pypi.org/project/thulium-htr/)

This notebook demonstrates how to analyze recognition errors.

**Topics covered:**
- Error summarization
- Top-K error analysis
- Character-level debugging
- Common error patterns

## Installation

In [ ]:
# Install Thulium (uncomment in Colab)
# !pip install thulium-htr -q

In [ ]:
import thulium
print(f"Thulium version: {thulium.__version__}")

## 1. Sample Data

Let's create some example predictions and ground truth:

In [ ]:
predictions = [
    "The quick brown fox",
    "jumps over the lazy dog",
    "Hello World",
    "Testing 123",
]

ground_truth = [
    "The quick brown fox",  # Correct
    "jumps over the lazy dog",  # Correct
    "Hallo World",  # Error: Hello -> Hallo
    "Testing 1234",  # Error: missing 4
]

## 2. Error Summary

In [ ]:
from thulium.xai.error_analysis import ErrorAnalyzer

# Get aggregate metrics
metrics = ErrorAnalyzer.summarize_metrics(predictions, ground_truth)

print("Summary Metrics:")
print(f"  CER: {metrics['CER']:.2%}")
print(f"  WER: {metrics['WER']:.2%}")
print(f"  SER: {metrics['SER']:.2%}")

## 3. Top-K Errors

Find the most frequent errors:

In [ ]:
top_errors = ErrorAnalyzer.analyze_top_k_errors(
    predictions, 
    ground_truth, 
    k=10
)

print("\nTop Errors:")
for i, error in enumerate(top_errors, 1):
    print(f"{i}. '{error['ground_truth']}' → '{error['prediction']}'")
    print(f"   Count: {error['count']}, CER: {error['cer']:.2%}")

## 4. Common Error Patterns

| Pattern | Example | Cause | Fix |
|---------|---------|-------|-----|
| Similar chars | o → 0, l → 1 | Font ambiguity | More training data |
| Missing punct | . omitted | Bias in training | Augment punctuation |
| Wrong case | A → a | Case sensitivity | Normalize case |
| Extra spaces | word  word | Segmentation | Better preprocessing |

## 5. Character-Level Analysis

In [ ]:
from thulium.evaluation.metrics import cer

# Analyze a specific error
ref = "hello"
hyp = "hallo"

error = cer(ref, hyp)
print(f"'{ref}' → '{hyp}'")
print(f"CER: {error:.2%}")
print(f"Error: 'e' was substituted with 'a'")

## 6. CLI Error Analysis

```bash
# Analyze errors from command line
thulium analyze predictions.txt ground_truth.txt \
    --top-k 50 \
    --format markdown \
    --output error_report.md
```

## Next Steps

- [Attention Visualization](../docs/xai/attention_and_saliency.md) - See what the model looks at
- [Training Guide](../docs/training/training_guide.md) - Improve model accuracy
- [Robustness Testing](../docs/evaluation/robustness.md) - Test noise resistance